In [ ]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

2023-07-10 14:31:21.247317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:31:21.400214: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 14:31:22.096010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 14:31:22.096067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [ ]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
len(real_speakers + fake_speakers)

411

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split(balanced=False)

149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
voices_to_remove =['Adam', 'Antoni', 'FAEM0', 'FAJW0', 'FALK0', 'FALR0', 'FAPB0', 'FBAS0', 'FBCG1', 'FBCH0', 'FBJL0', 'FBLV0', 'FBMH0', 'FCAG0', 'FCAJ0', 'FCDR1', 'FCEG0', 'FCJF0', 'FCJS0', 'FCLT0', 'FCMG0', 'FCMM0']
df_2 = timit_data_loader.generate_split_speaker_test(voices_to_remove)

149
Params: 149 different phrases
# of Train instances: 813
# of Dev instances: 271
# of Test instances: 170


In [ ]:
df_2[df_2['path'] == '/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Bella_SX328.wav']

,type,id,architecture,orig_path,label,multiclass_label,path,speaker


In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
voices_to_remove =['Adam', 'Antoni', 'FAEM0', 'FAJW0', 'FALK0', 'FALR0', 'FAPB0', 'FBAS0', 'FBCG1', 'FBCH0', 'FBJL0', 'FBLV0', 'FBMH0', 'FCAG0', 'FCAJ0', 'FCDR1', 'FCEG0', 'FCJF0', 'FCJS0', 'FCLT0', 'FCMG0', 'FCMM0']
df_3 = timit_data_loader.generate_split_speaker_test(voices_to_remove)
df_4 = timit_data_loader.generate_split_speaker_test(voices_to_remove)

149
Params: 149 different phrases
# of Train instances: 813
# of Dev instances: 271
# of Test instances: 170
149
Params: 149 different phrases
# of Train instances: 817
# of Dev instances: 272
# of Test instances: 163


In [ ]:
df_4

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,train,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
3,train,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
4,train,5,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MCAL0
...,...,...,...,...,...,...,...,...
1247,train,1520,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1248,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/R...,Rachel
1249,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Arnold
1250,test,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni


In [ ]:
df_3

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,train,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,train,5,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MCAL0
...,...,...,...,...,...,...,...,...
1249,dev,1520,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Biden
1250,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/J...,Josh
1251,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/R...,Rachel
1252,dev,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
for path in df['path']:
    if path not in list(df_2['path']):
        print(path, ' in df1 but not 2')

/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MMJB1_SX238.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FAEM0_SX132.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MRPC1_SX132.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MRLD0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MJRH1_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MKAJ0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MLJH0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FJRP1_SX82.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FNKL0_SX82.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MSMC0_SX197.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MDLB0_SX406.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FGCS0_SX406.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MGAK0_SX4

In [ ]:
df

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,dev,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
...,...,...,...,...,...,...,...,...
1521,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni
1522,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1523,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Adam
1524,train,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FKLC1_SX238.WAV'

In [ ]:
path = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'
len(list(os.listdir(path)))

1527

In [ ]:
missing = []
for file in df['path']:
    if os.path.isfile(file) is False:
        missing.append(file)
print(len(missing))
    

/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Domi_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Josh_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX132.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Arnold_SX132.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Arnold_SX154.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX154.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Josh_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Bella_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX433.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Domi_SX197.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX197.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Elli_SX406.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX178.wav
/home/ubuntu/data/TIMIT_and_ElevenL

In [ ]:

import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)


Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FKLC1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FKLC1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MHRM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MHRM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MRJM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MRJM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FDFB0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FDFB0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MMJB1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MMJB1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MCAL0_SX238.WAV to /home/ubuntu/data/TI

In [ ]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX176/fake/Antoni_SX176.wav'

In [ ]:
#df[df['label'] == 1]

In [ ]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [ ]:
### End of timit additions ###

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    if data_df is None:
        data_df = loader.generate_split_speaker_test(speakers_to_remove)
        data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/DATA_DF_{name}.csv')
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [ ]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
#run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_single_fold_TEST', loader=timit_data_loader)

# Multiple results run
results = []
counter = 0 

for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()


In [ ]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/16KHz_ElevenLabs_TIMIT_k_fold_regenerated.csv')

# RUN DOWN TO HERE! ^ 

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [ ]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

2023-07-10 14:31:21.247317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:31:21.400214: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 14:31:22.096010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 14:31:22.096067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [ ]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
len(real_speakers + fake_speakers)

411

In [ ]:
df

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,dev,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
...,...,...,...,...,...,...,...,...
1521,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni
1522,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1523,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Adam
1524,train,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FKLC1_SX238.WAV'

In [ ]:
path = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'
len(list(os.listdir(path)))

1527

In [ ]:

import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)


Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FKLC1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FKLC1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MHRM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MHRM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MRJM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MRJM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FDFB0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FDFB0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MMJB1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MMJB1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MCAL0_SX238.WAV to /home/ubuntu/data/TI

In [ ]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX176/fake/Antoni_SX176.wav'

In [ ]:
#df[df['label'] == 1]

In [ ]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [ ]:
### End of timit additions ###

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    if data_df is None:
        data_df = loader.generate_split_speaker_test(speakers_to_remove)
        data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/DATA_DF_{name}.csv')
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [ ]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
#run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_single_fold_TEST', loader=timit_data_loader)

# Multiple results run
results = []
counter = 0 

for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()


In [ ]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/16KHz_ElevenLabs_TIMIT_k_fold_regenerated.csv')

# RUN DOWN TO HERE! ^ 

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [ ]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

2023-07-10 14:31:21.247317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:31:21.400214: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 14:31:22.096010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 14:31:22.096067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [ ]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
len(real_speakers + fake_speakers)

411

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split(balanced=False)

149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
voices_to_remove =['Adam', 'Antoni', 'FAEM0', 'FAJW0', 'FALK0', 'FALR0', 'FAPB0', 'FBAS0', 'FBCG1', 'FBCH0', 'FBJL0', 'FBLV0', 'FBMH0', 'FCAG0', 'FCAJ0', 'FCDR1', 'FCEG0', 'FCJF0', 'FCJS0', 'FCLT0', 'FCMG0', 'FCMM0']
df_2 = timit_data_loader.generate_split_speaker_test(voices_to_remove)

149
Params: 149 different phrases
# of Train instances: 813
# of Dev instances: 271
# of Test instances: 170


In [ ]:
df_2[df_2['path'] == '/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Bella_SX328.wav']

,type,id,architecture,orig_path,label,multiclass_label,path,speaker


In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
voices_to_remove =['Adam', 'Antoni', 'FAEM0', 'FAJW0', 'FALK0', 'FALR0', 'FAPB0', 'FBAS0', 'FBCG1', 'FBCH0', 'FBJL0', 'FBLV0', 'FBMH0', 'FCAG0', 'FCAJ0', 'FCDR1', 'FCEG0', 'FCJF0', 'FCJS0', 'FCLT0', 'FCMG0', 'FCMM0']
df_3 = timit_data_loader.generate_split_speaker_test(voices_to_remove)
df_4 = timit_data_loader.generate_split_speaker_test(voices_to_remove)

149
Params: 149 different phrases
# of Train instances: 813
# of Dev instances: 271
# of Test instances: 170
149
Params: 149 different phrases
# of Train instances: 817
# of Dev instances: 272
# of Test instances: 163


In [ ]:
df_4

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,train,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
3,train,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
4,train,5,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MCAL0
...,...,...,...,...,...,...,...,...
1247,train,1520,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1248,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/R...,Rachel
1249,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Arnold
1250,test,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni


In [ ]:
df_3

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,train,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,train,5,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MCAL0
...,...,...,...,...,...,...,...,...
1249,dev,1520,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Biden
1250,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/J...,Josh
1251,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/R...,Rachel
1252,dev,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
for path in df['path']:
    if path not in list(df_2['path']):
        print(path, ' in df1 but not 2')

/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MMJB1_SX238.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FAEM0_SX132.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MRPC1_SX132.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MRLD0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MJRH1_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MKAJ0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MLJH0_SX154.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FJRP1_SX82.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FNKL0_SX82.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MSMC0_SX197.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MDLB0_SX406.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FGCS0_SX406.WAV  in df1 but not 2
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MGAK0_SX4

In [ ]:
df

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,dev,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
...,...,...,...,...,...,...,...,...
1521,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni
1522,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1523,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Adam
1524,train,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FKLC1_SX238.WAV'

In [ ]:
path = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'
len(list(os.listdir(path)))

1527

In [ ]:
missing = []
for file in df['path']:
    if os.path.isfile(file) is False:
        missing.append(file)
print(len(missing))
    

/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Domi_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Josh_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX238.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX132.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Arnold_SX132.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Arnold_SX154.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX154.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Josh_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Bella_SX82.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX433.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Domi_SX197.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Adam_SX197.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Elli_SX406.wav
/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/Biden_SX178.wav
/home/ubuntu/data/TIMIT_and_ElevenL

In [30]:
import pathlib
all_wav_files = pathlib.Path('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
all_wav_files = list(all_wav_files.rglob("*.wav")) + list(all_wav_files.rglob("*.WAV"))
all_wav_files = [str(file) for file in all_wav_files]

In [ ]:

import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)


Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FKLC1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FKLC1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MHRM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MHRM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MRJM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MRJM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FDFB0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FDFB0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MMJB1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MMJB1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MCAL0_SX238.WAV to /home/ubuntu/data/TI

In [ ]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX176/fake/Antoni_SX176.wav'

In [ ]:
#df[df['label'] == 1]

In [ ]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [ ]:
### End of timit additions ###

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    if data_df is None:
        data_df = loader.generate_split_speaker_test(speakers_to_remove)
        data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/DATA_DF_{name}.csv')
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [ ]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
#run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_single_fold_TEST', loader=timit_data_loader)

# Multiple results run
results = []
counter = 0 

for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()


In [ ]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/16KHz_ElevenLabs_TIMIT_k_fold_regenerated.csv')

# RUN DOWN TO HERE! ^ 

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [ ]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

2023-07-10 14:31:21.247317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:31:21.400214: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 14:31:22.096010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 14:31:22.096067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [ ]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


149
Params: 149 different phrases
# of Train instances: 915
# of Dev instances: 305
# of Test instances: 306


In [ ]:
len(real_speakers + fake_speakers)

411

In [ ]:
df

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,dev,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FKLC1
1,train,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MHRM0
2,train,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRJM0
3,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/F...,FDFB0
4,dev,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMJB1
...,...,...,...,...,...,...,...,...
1521,train,1521,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Antoni
1522,train,1522,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Bella
1523,train,1523,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/A...,Adam
1524,train,1524,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/O...,Obama


In [ ]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/FKLC1_SX238.WAV'

In [ ]:
path = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'
len(list(os.listdir(path)))

1527

In [ ]:

import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)


Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FKLC1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FKLC1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MHRM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MHRM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MRJM0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MRJM0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/FDFB0_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/FDFB0_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MMJB1_SX238.WAV to /home/ubuntu/data/TIMIT_and_ElevenLabs/Original/MMJB1_SX238.WAV
Successfully copied /home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX238/real/MCAL0_SX238.WAV to /home/ubuntu/data/TI

In [ ]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX176/fake/Antoni_SX176.wav'

In [ ]:
#df[df['label'] == 1]

In [ ]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [ ]:
### End of timit additions ###

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    if data_df is None:
        data_df = loader.generate_split_speaker_test(speakers_to_remove)
        data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/DATA_DF_{name}.csv')
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [ ]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
#run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_single_fold_TEST', loader=timit_data_loader)

# Multiple results run
results = []
counter = 0 

for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()


In [ ]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/16KHz_ElevenLabs_TIMIT_k_fold_regenerated.csv')

# RUN DOWN TO HERE! ^ 

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss